In [73]:
import requests
import json
import pandas as pd 
import numpy as np

In [ ]:
launchpad_price_mint = {}

url = "http://api-mainnet.magiceden.dev/v2/launchpad/collections?offset=0&limit=500"
payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
response_json = response.json()

for n in range(0,len(response_json)):
    launchpad_price_mint.update(
        {
            response_json[n]['symbol']: response_json[n]['price']
        }
    )

In [71]:
collection_price_floor = {}

for key in launchpad_mint_price.keys():
    url = "http://api-mainnet.magiceden.dev/v2/collections/"+key+"/stats"
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    response_json = response.json()

    try:
        collection_price_floor.update(
            {
                response_json['symbol']: response_json['floorPrice']
            }
        )
    except:
        collection_price_floor.update(
            {
                response_json['symbol']: 0
            }
        )

In [88]:
collection_stats = {
    'symbol': list(launchpad_mint_price.keys()),
    'price_mint': list(launchpad_mint_price.values()),
    'price_floor': list(collection_price_floor.values())
}

In [105]:
df = pd.DataFrame(collection_stats)
df = df[df.price_floor > 0].reset_index(drop=True) # Excluding collections with no floor price
df = df[df.price_mint > 0].reset_index(drop=True) # Excluding collections with no mint price

df['price_floor'] = df.price_floor.apply(lambda x: x / (10**9)) # price floor to 9 decimals
df['roi_times'] = df.price_floor / df.price_mint
df.head()

,symbol,price_mint,price_floor,roi_times
0,solshades_vouchers,1.25,0.289,0.231200
1,infinity_serpents,1.00,0.790,0.790000
2,sovana,1.00,2.950,2.950000
3,metaheaven_keys,1.19,0.240,0.201681
4,balloonsville,1.00,3.100,3.100000


In [106]:
df.describe()

,price_mint,price_floor,roi_times
count,115.000000,115.000000,115.000000
mean,1.274574,2.713821,48.732493
std,0.648842,10.137526,467.679624
min,0.000010,0.009000,0.009000
25%,0.965000,0.190000,0.157222
50%,1.000000,0.680000,0.620000
75%,1.525000,1.686500,1.504167
max,4.200000,100.000000,5000.000000


In [120]:
def bin_mint(x):
    assigned_bin = ''

    if x < 1:
        assigned_bin = 'Less than 1'
    elif 1 <= x < 2:
        assigned_bin = '1-2'
    elif 2 <= x < 3:
        assigned_bin = '2-3'
    else:
        assigned_bin = 'More than 3'
    
    return assigned_bin

def bin_roi(x):
    assigned_bin = ''
    if x < 1:
        assigned_bin = 'Less than 1x Returns'
    elif 1 <= x < 5:
        assigned_bin = '1-5x Returns'
    elif 5 <= x < 10:
        assigned_bin = '5-10x Returns'
    elif 10 <= x < 20:
        assigned_bin = '10-20x Returns'
    else:
        assigned_bin = 'More than 20x Returns'

    return assigned_bin

df['bin_price_mint'] = df.price_mint.apply(bin_mint)
df['bin_returns'] = df.roi_times.apply(bin_roi)

In [121]:
df.head()

,symbol,price_mint,price_floor,roi_times,bin_price_mint,bin_returns
0,solshades_vouchers,1.25,0.289,0.231200,1-2,Less than 1x Returns
1,infinity_serpents,1.00,0.790,0.790000,1-2,Less than 1x Returns
2,sovana,1.00,2.950,2.950000,1-2,1-5x Returns
3,metaheaven_keys,1.19,0.240,0.201681,1-2,Less than 1x Returns
4,balloonsville,1.00,3.100,3.100000,1-2,1-5x Returns


In [173]:
binned_df = df.groupby(['bin_price_mint','bin_returns']).symbol.count().reset_index()

pivoted = binned_df.pivot(index='bin_price_mint',columns='bin_returns',values='symbol').reset_index()
col_order = ['bin_price_mint','Less than 1x Returns','1-5x Returns','5-10x Returns','10-20x Returns','More than 20x Returns']
pivoted = pivoted[col_order]
pivoted = pivoted.rename(columns={
        'bin_price_mint': 'Mint Price (SOL)'
    },
    index={
        0: 1,
        1: 2,
        2: 0,
        3: 3
    })
# pivoted.index.rename('index',inplace=True)
pivoted.sort_index(ascending=True, inplace=True)
pivoted.set_index('Mint Price (SOL)', inplace=True)
pivoted.columns.name = None

pivoted

,Less than 1x Returns,1-5x Returns,5-10x Returns,10-20x Returns,More than 20x Returns
Mint Price (SOL),,,,,
Less than 1,25.0,8.0,NaN,2.0,2.0
1-2,45.0,13.0,NaN,NaN,NaN
2-3,8.0,9.0,1.0,1.0,NaN
More than 3,NaN,1.0,NaN,NaN,NaN
